In [1]:
import pandas as pd
import numpy as np

## Training
- you need to modify `/data/home/wangys/LLaMA-Factory/src/llamafactory/data/loader.py` as below:
- from datasets import DatasetDict, load_dataset, load_from_disk, Dataset
- import pandas as pd
- import numpy as np
- for function `def _load_single_dataset`, please insert the following code block after the final `else`:
```
    else:
        print(data_path,data_name,data_dir,data_files,dataset_attr.split,model_args.cache_dir,model_args.trust_remote_code)
        if data_args.tokenized_path is not None:
            dataset = load_from_disk(data_args.tokenized_path)
            print('load pre-defined arrow')
        elif data_args.train_file_path is not None:
            data_files = data_args.train_file_path.split(',')
            df = pd.DataFrame()
            for data_file in data_files:   
                df_current = pd.read_json(data_file)
                df_current['ids'] = df_current.index
                df = pd.concat([df,df_current])
            dataset = Dataset.from_pandas(df)
            print('loading from json file')
        else:           
            try:
                df = pd.DataFrame()
                for data_file in data_files:   
                    df_current = pd.read_json(data_file)
                    df = pd.concat([df,df_current])
                dataset = Dataset.from_pandas(df)
                print('loading from pandas')
            except:
                dataset = load_dataset(
                    path=data_path,
                    name=data_name,
                    data_dir=data_dir,
                    data_files=data_files,
                    split=dataset_attr.split,
                    cache_dir=model_args.cache_dir,
                    token=model_args.hf_hub_token,
                    streaming=data_args.streaming,
                    num_proc=data_args.preprocessing_num_workers,
                    trust_remote_code=model_args.trust_remote_code,
                )
        print('load_dataset finished')

    if dataset_attr.num_samples is not None and not data_args.streaming:
        target_num = dataset_attr.num_samples
        indexes = np.random.permutation(len(dataset))[:target_num]  # all samples should be included
        target_num -= len(indexes)
        if target_num > 0:
            expand_indexes = np.random.choice(len(dataset), target_num)
            indexes = np.concatenate((indexes, expand_indexes), axis=0)

        assert len(indexes) == dataset_attr.num_samples, "Sample num mismatched."
        dataset = dataset.select(indexes)
        logger.info_rank0(f"Sampled {dataset_attr.num_samples} examples from dataset {dataset_attr}.")

    if data_args.max_samples is not None:  # truncate dataset
        max_samples = min(data_args.max_samples, len(dataset))
        dataset = dataset.select(range(max_samples))

    return align_dataset(dataset, dataset_attr, data_args, training_args)
```
This can skip huggingface-dataset check, and directly load the `json` file locally

- also modify `src/llamafactory/hparams/data_args.py` with the following args:
```
    train_file_path: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "json path for training file. "
            )
        },
    )
```

## OA-Mine dataset


In [2]:
AVE_OA_Mine_train = '/data/home/wangys/MELD/dataset/AVE/oa_mine_train_small.json'
AVE_OA_Mine_test = '/data/home/wangys/MELD/dataset/AVE/oa_mine_test_small.json'
AVE_OA_Mine_ckpt = ''

In [4]:
train = pd.read_json(AVE_OA_Mine_train)
train

,instruction,input,output
0,You are a world-class expert for extracting in...,,"{""Brand"": ""Retro Clean""}"
1,You are a world-class expert for extracting in...,,"{""Specific uses"": ""Cleaning Solution""}"
2,You are a world-class expert for extracting in...,,"{""Item form"": ""Bag""}"
3,You are a world-class expert for extracting in...,,"{""Brand"": ""Shout""}"
4,You are a world-class expert for extracting in...,,"{""Specialty"": ""Triple Acting""}"
...,...,...,...
1438,You are a world-class expert for extracting in...,,"{""Brand"": ""Mommy's Bliss""}"
1439,You are a world-class expert for extracting in...,,"{""Specialty"": ""No Artificial Color""}"
1440,You are a world-class expert for extracting in...,,"{""Item form"": ""Drops""}"
1441,You are a world-class expert for extracting in...,,"{""Supplement type"": ""Vitamin D""}"


In [ ]:
ave_train = pd.read_csv('inference_Transfer_ER/init--oa_mine-test.csv',index_col=0)
import ast
def AST(row):
    output = row['output'].strip()
    predict = row['predict'].strip()
    output_item = list(ast.literal_eval(output).values())[0]
    predict_item = list(ast.literal_eval(predict).values())[0]
    row['output_item'] = output_item.lower()
    row['predict_item'] = predict_item.lower()
    return row
ave_train = ave_train.apply(AST,axis=1)
len(ave_train[ave_train['output_item']==ave_train['predict_item']]) / len(ave_train)

0.8049775601795186

: 

## Data Imputation(DI)

In [32]:
DI_restaurant_train = '/data/home/wangys/MELD/dataset/DI/walmart_train_output_wide.json'
DI_restaurant_test = '/data/home/wangys/MELD/dataset/DI/walmart_test_output_wide.json'

In [33]:
train = pd.read_json(DI_restaurant_train)
train.drop_duplicates()

,instruction,input,output
0,You are an expert in attribute extraction from...,,{'brand': 'd-link'}
1,You are an expert in attribute extraction from...,,{'category': 'garden - general'}
2,You are an expert in attribute extraction from...,,{'brand': 'd-link'}
3,You are an expert in attribute extraction from...,,{'category': 'garden - general'}
4,You are an expert in attribute extraction from...,,{'brand': 'gear head'}
...,...,...,...
6934,You are an expert in attribute extraction from...,,{'brand': '3m'}
6935,You are an expert in attribute extraction from...,,{'brand': 'motorola'}
6936,You are an expert in attribute extraction from...,,{'category': 'walkie-talkie/frs'}
6937,You are an expert in attribute extraction from...,,{'brand': 'motorola'}


In [18]:
raw_train = pd.read_csv('/data/home/wangys/MoE-Example/DI/restaurant/restaurant_train_few.csv',index_col=0)
raw_train

,id,name,addr,city,phone,type,class,city_backup
303,303,marichu,' 342 e. 46th st. between 1st and 2nd aves . ',` new york ',212/370 -1866,french,302,NaN
124,124,` california pizza kitchen ',' 207 s. beverly dr. ',` los angeles ',310/275 -1101,californian,121,NaN
78,78,` buckhead diner ',' 3073 piedmont road ',atlanta,404/262 -3336,american,78,NaN
297,297,` lucky strike ','59 grand st. between wooster st. and w. broad...,` new york ',212/941 -0479,` or 212/941 -0772 american ',296,NaN
259,259,global,'33 93 2nd ave. between 5th and 6th sts . ',` new york ',212/477 -8427,american,257,NaN
...,...,...,...,...,...,...,...,...
424,424,` bally \ 's big kitchen ',' 3645 las vegas blvd. s ',` las vegas ',702/739 -4111,buffets,424,NaN
590,57,` ruby \ 's ( la ) ','45 s. fair oaks ave. ',pasadena,818-796-7829,diners,591,` los angeles '
273,273,` jean lafitte ','68 w. 58th st. ',` new york ',212/751 -2323,french,272,NaN
24,24,aquavit,'13 w. 54th st. ',` new york ',212/307 -7311,continental,24,NaN


## raw dataset for restaurant


In [29]:
index = np.load('/data/home/wangys/MoE-Example/DI/amazon/amazon_test_few_index.npy')
index.shape

(4003,)

In [27]:
# raw_data = '/data/home/wangys/MoE-Example/DI/restaurant/restaurant_all.csv'
# raw_data = '/data/home/wangys/MoE-Example/DI/amazon/amazon_all_filter_train_few.csv'
raw_data = '/data/home/wangys/MoE-Example/DI/amazon/amazon_all_filter_train.csv'
raw_data_df = pd.read_csv(raw_data,index_col=0)
raw_data_df

,id,title,category,brand,modelno,price,query,category_predict,brand_predict
0,0,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132,12.65,"{'title': 'koss eq50 3-band stereo equalizer',...","['stereo amplifiers', 'home audio crossovers',...","['shure', 'switcheasy', 'jvc', 'koss', 'meelec..."
1,1,kodak black ink cartridge 10b 1163641,inkjet printer ink,kodak,1163641,10.28,{'title': 'kodak black ink cartridge 10b 11636...,"['inkjet printers', 'computer accessories', 'i...","['oki data', 'ocinkjet', 'kodak', 'lexmark', '..."
2,2,kingston 128mx64 pc2700 compaq evo d320 ktc-d3...,computers accessories,kingston,ktc-d320 / 1g,33.75,{'title': 'kingston 128mx64 pc2700 compaq evo ...,"['cpu processors', 'computer accessories', 'de...","['compaq', 'lenovo', 'kingston', 'mushkin', 's..."
3,3,kinamax ms-ues2 mini high precision usb 3-butt...,mice,kinamax,ms-ues2,6.99,{'title': 'kinamax ms-ues2 mini high precision...,"['controllers', 'keyboard mouse combos', 'gami...","['neewer', 'keytronicems', 'mad catz inc', 'at..."
4,4,kensington k72349us wireless mouse for netbooks,mice,NaN,k72349us,24.00,{'title': 'kensington k72349us wireless mouse ...,"['mice', 'laminators', 'touch pads', 'keyboard...","['kensington', 'chester creek', 'fellowes', 'a..."
...,...,...,...,...,...,...,...,...,...
22067,22067,memorex mini dvd rw 10 pack,NaN,memorex,32020016369,14.81,"{'title': 'memorex mini dvd rw 10 pack', 'bran...","['dvd rw discs', 'dvd-rw discs', 'dvd r discs'...","['memtek', 'memorex', 'imation', 'rolls', 'qms']"
22068,22068,maxell dvd-r double sided 2.8 gb 3pack media f...,dvd-r discs,maxell,567647,2.99,{'title': 'maxell dvd-r double sided 2.8 gb 3p...,"['cd-r discs', 'car audio video', 'dvd-r discs...","['maxram', 'maxell', 'imation', 'lexar', 'memt..."
22070,22070,da-lite stand master i - cart for projector,projection screens,da-lite,i,329.95,{'title': 'da-lite stand master i - cart for p...,"['projection screens', 'video projector access...","['da-lite', 'da-lite screen', 'master caster',..."
22072,22072,link depot fan led fan 120mm green,computers accessories,link depot,fan-4led-120gn,29.23,{'title': 'link depot fan led fan 120mm green'...,"['office electronics', 'computers accessories'...","['link depot', 'multilink', 'interlink', 'db',..."


## Schema Matching

In [34]:
train = pd.read_json('/data/home/wangys/MELD/dataset/SM/CMS_train_few_output.json')

In [2]:
RE_index = np.load('/data/home/wangys/bck_16_229/LLaMA-Efficient-Tuning-main/RE_sample_10_index.npy')


## ER

In [2]:
ER_train_file_path = '/data/home/wangys/transfer-er/process_data/amazon-google/amazon_google_train_filter.csv'

In [5]:
ER_train = pd.read_csv(ER_train_file_path,index_col=0)

### Amazon-Google

In [9]:
ER_train[ER_train['label']==1].iloc[:,:3].drop_duplicates().to_csv('../MELD/raw_dataset/ER/Amazon-Google/index.csv')

In [14]:
ER_train_file_path = '/data/home/wangys/transfer-er/process_data/amazon-google/amazon_google_test_filter.csv'
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
ER_train

,ltable_id,rtable_id,label,l_entity,r_entity,l_entity_diff,r_entity_diff
0,393,831,0,{'title': 'microsoft visual studio test agent ...,{'title': 'individual software professor teach...,{'title': 'microsoft visual studio test agent ...,{'title': 'individual software professor teach...
1,559,324,0,"{'title': 'adobe dreamweaver cs3 [ mac ]', 'ma...","{'title': 'adobe cs3 web premium', 'manufactur...","{'title': 'adobe dreamweaver cs3 [ mac ]', 'ma...","{'title': 'adobe cs3 web premium', 'manufactur..."
2,558,3023,0,"{'title': 'instant immersion spanish 2.0', 'ma...","{'title': 'instant immers japanese dlx 2 .', '...","{'title': 'instant immersion spanish 2.0', 'ma...","{'title': 'instant immers japanese dlx 2 .', '..."
3,762,1618,0,{'title': 'i transfer dvds 2 ipod edition ( wi...,{'title': 'ghost recon gold edition ( dvd-rom ...,{'title': 'i transfer dvds 2 ipod edition ( wi...,{'title': 'ghost recon gold edition ( dvd-rom ...
4,1262,2860,0,{'title': 'adobe flash pro cs3 upgrade [ mac ]...,"{'title': 'adobe illustrator cs3 ( mac )', 'ma...",{'title': 'adobe flash pro cs3 upgrade [ mac ]...,"{'title': 'adobe illustrator cs3 ( mac )', 'ma..."
...,...,...,...,...,...,...,...
2288,646,2501,0,"{'title': 'instant immersion german audio', 'm...","{'title': 'instant immers french dlx 2', 'manu...","{'title': 'instant immersion german audio', 'm...","{'title': 'instant immers french dlx 2', 'manu..."
2289,642,1927,0,"{'title': 'adobe flash pro cs3 [ mac ]', 'manu...",{'title': '38039295dm adobe flash cs3 professi...,"{'title': 'adobe flash pro cs3 [ mac ]', 'manu...",{'title': '38039295dm adobe flash cs3 professi...
2290,342,570,0,"{'title': ""let 's learn about time & date"", 'm...","{'title': ""it 's about time products learn the...","{'title': ""let 's learn about time & date"", 'm...","{'title': ""it 's about time products learn the..."
2291,1241,3092,0,"{'title': 'quicken legal business pro 2007', '...","{'title': 'quickbooks ( r ) pro 2007', 'manufa...","{'title': 'quicken legal business pro 2007', '...","{'title': 'quickbooks ( r ) pro 2007', 'manufa..."


In [16]:
ER_train.iloc[:,:5].to_csv('../MELD/raw_dataset/ER/Amazon-Google/test.csv')

### Walmart-Amazon

In [20]:
ER_train_file_path = '/data/home/wangys/transfer-er/process_data/walmart-amazon/train.csv'
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates().to_csv('../MELD/raw_dataset/ER/Walmart-Amazon/index.csv')

In [22]:
ER_train.to_csv('../MELD/raw_dataset/ER/Walmart-Amazon/train.csv')

In [24]:
ER_train_file_path = '/data/home/wangys/transfer-er/process_data/walmart-amazon/test_align.csv'
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
ER_train.to_csv('../MELD/raw_dataset/ER/Walmart-Amazon/test.csv')

### Abt-Buy

In [26]:
ER_train_file_path = '/data/home/wangys/transfer-er/process_data/ant-buy/train.csv'
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates()
ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates().to_csv('../MELD/raw_dataset/ER/Abt-Buy/index.csv')

In [28]:
ER_train_file_path = '/data/home/wangys/transfer-er/process_data/ant-buy/train.csv'
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
# ER_train
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates()
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates().to_csv('../MELD/raw_dataset/ER/Abt-Buy/index.csv')
ER_train.to_csv('../MELD/raw_dataset/ER/Abt-Buy/train.csv')

In [31]:
ER_train_file_path = '/data/home/wangys/transfer-er/process_data/ant-buy/test.csv'
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
# ER_train.iloc[:,:5]
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates()
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates().to_csv('../MELD/raw_dataset/ER/Abt-Buy/index.csv')
ER_train.iloc[:,:5].to_csv('../MELD/raw_dataset/ER/Abt-Buy/test.csv')

### WDC-All

In [33]:
ER_train_file_path = '/data/home/wangys/transfer-er/process_data/wdc-all/train.csv'
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates()
ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates().to_csv('../MELD/raw_dataset/ER/WDC-All/index.csv')

In [35]:
ER_train_file_path = '/data/home/wangys/transfer-er/process_data/wdc-all/train.csv'
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
# ER_train
ER_train.to_csv('../MELD/raw_dataset/ER/WDC-All/train.csv')
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates()
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates().to_csv('../MELD/raw_dataset/ER/WDC-All/index.csv')

In [38]:
ER_train_file_path = '/data/home/wangys/transfer-er/process_data/wdc-all/wdc_all/dpo_test.csv'
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
ER_train.iloc[:,:5].to_csv('../MELD/raw_dataset/ER/WDC-All/test.csv')
# ER_train.to_csv('../MELD/raw_dataset/ER/WDC-All/train.csv')
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates()
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates().to_csv('../MELD/raw_dataset/ER/WDC-All/index.csv')

## Semi-Text-Watch

In [51]:
ER_train_file_path = '/data/home/wangys/LLM_ER/process_data/semi-text-w/train_add_unique.csv'
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
# ER_train[ER_train['label']==1].iloc[:,[0,1,2]].drop_duplicates()
ER_train
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates().to_csv('../MELD/raw_dataset/ER/WDC-All/index.csv')

,ltable_id,rtable_id,label,l_entity,r_entity
0,8810,1595,0,"{'/productID': '[10061229]', 'brand': '"" ...","{'title': 'Cartier', 'brand': 'Cartier', 'colo..."
1,0,3086,0,"{'/productID': '[10061229]', 'brand': '"" ...","{'title': 'Cartier', 'brand': 'Cartier', 'colo..."
4,0,4783,0,"{'/productID': '[10061229]', 'brand': '"" ...","{'title': 'Cartier', 'brand': 'Cartier', 'colo..."
5,8001,7811,0,"{'/productID': '[10061229]', 'brand': '"" ...","{'title': ' ""Freja Reloj para mujer""@es ""Skag..."
7,0,1329,0,"{'/productID': '[10061229]', 'brand': '"" ...","{'title': '', 'brand': 'Calvin Klein', 'color'..."
...,...,...,...,...,...
9153,3768,5307,1,"{'/productID': '[srp639k1]', '/sku': '[srp639k...","{'title': ' ""Seiko SRP639K1 Prospex Sea watch..."
9157,6545,5307,1,"{'/productID': '[srp639k1]', '/sku': '[srp639k...","{'title': ' ""Seiko SRP639K1 Prospex Sea watch..."
9160,1531,4928,1,"{'/mpn': '[90048731]', 'brand': '""Polar""@en-US...",{'title': 'Polar USA Unisex FT4 Green/Green Wa...
9176,2252,2463,0,"{'/mpn': '[w51011q3]', 'brand': 'Cartier', 'ca...",{'title': 'Cartier W51002Q3 Tank Francaise Lar...


In [54]:
ER_train_file_path = '/data/home/wangys/LLM_ER/process_data/semi-text-w/test_EM_w.csv'
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
# ER_train[ER_train['label']==1].iloc[:,[0,1,2]].drop_duplicates()
ER_train.to_csv('../MELD/raw_dataset/ER/Semi-Text-Watch/test.csv')
# ER_train.to_csv('../MELD/raw_dataset/ER/Semi-Text-Watch/train.csv')
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates().to_csv('../MELD/raw_dataset/ER/WDC-All/index.csv')

## Semi-Text-Computer

In [56]:
ER_train_file_path = '/data/home/wangys/LLM_ER/process_data/semi-text-c/train_add_unique.csv'
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
# ER_train[ER_train['label']==1].iloc[:,[0,1,2]].drop_duplicates()
ER_train.to_csv('../MELD/raw_dataset/ER/Semi-Text-Computer/train.csv')
# ER_train[ER_train['label']==1].iloc[:,[0,1,-1]].drop_duplicates().to_csv('../MELD/raw_dataset/ER/WDC-All/index.csv')

In [77]:
Dataset_Name = 'Semi-Text-Computer'
ER_train_file_path = '../MELD/raw_dataset/ER/{}/train.csv'.format(Dataset_Name)
ER_train = pd.read_csv(ER_train_file_path,index_col=0)
ER_train
eval(ER_train.iloc[0,4]).keys()
# ER_train.iloc[0,2].replace('\n','')

dict_keys(['title', 'category', 'subcategory', 'brand', 'sku', 'type', 'description'])

: 

In [ ]:
find Expert/AVE \( -type f \( -name "all_results.json" -o -name "train_results.json" -o -name "trainer_log.jsonl" -o -name "trainer_state.json" -o -name "training_loss.png" \) -delete \) -o \( -type d -name "*checkpoint*" -exec rm -r {} + \)

In [ ]:
CUDA_VISIBLE_DEVICES=6,7 python vllm_inference_mistral_api.py --model_path /data/home/wangys/model/Mistral-7B-Instruct-v0.2 --directory lora_weight/Expert/ER/wdc_all-MoE-Add --gpu_num 2 --file dataset/ER/wdc_all_test_output.json --max_token 32 --json